In [1]:
from peewee import *
from datetime import date
import pandas as pd
import re
from ast import literal_eval
import seaborn as sns
from matplotlib import pyplot as plt

DB_NAME = 'HorsingAround'
USER = 'maartjehuveneers'
PASSWORD = 'p4RwxJCchw7Ljqhv'
HOST = 'www.jacobkamminga.nl'
PORT = 3306


# Establish connection to database
def connect(db_name, username, password, host, port):
    db = MySQLDatabase(db_name, user=username, password=password, host=host, port=port)
    db.connect()
    print("Connected to: " + host + ":" + str(port))
    return db


def save_experiment(Experiment, uid, horse, acc, balanced_acc, f_score_avg, mcc_score, recall, matrix, params, desc):
    Experiment.create(key=uid,
                      username=USER,
                      test_horse=horse,
                      date=date.today(),
                      accuracy_experiment=acc,
                      balanced_accuracy_experiment=balanced_acc,
                      fscore=f_score_avg,
                      mcc=mcc_score,
                      recall=recall,
                      confusion_matrix=matrix,
                      parameters=params,
                      description=desc)

def save_activity(Activity, uid, horse, activity, matrix, index, recall, precision):
    TP = matrix[index][index]
    FN = matrix[index].sum() - TP
    FP = 0
    for j in range(0, len(matrix)):
        FP += matrix[j][index]
    FP = FP - TP
    TN = matrix.sum() - TP - FN - FP

    specificity = TN/(TN+FP)
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    #save results per activity per horse in the database
    Activity.create(key=uid, 
                    test_horse=horse, 
                    activity=activity, 
                    accuracy_activity=accuracy, 
                    recall_activity=recall, 
                    specificity=specificity, 
                    precision=precision, 
                    TP=TP, 
                    TN=TN,
                    FP=FP,
                    FN=FN)

def get_classes(db):
    class Experiment(Model):
        key = UUIDField()
        username = TextField()
        test_horse = TextField()
        date = DateField()
        accuracy_experiment = FloatField()
        balanced_accuracy_experiment = FloatField()
        fscore = FloatField()
        mcc = FloatField()
        recall = FloatField()
        confusion_matrix = BlobField()
        parameters = TextField()
        description = TextField()

        class Meta:
            database = db


    class Activity(Model):
        key = UUIDField()
        test_horse = TextField()
        activity = TextField()
        accuracy_activity = FloatField()
        recall_activity = FloatField()
        specificity = FloatField()
        precision = FloatField()
        TP = IntegerField()
        TN = IntegerField()
        FP = IntegerField()
        FN = IntegerField()

        class Meta:
            database = db
            
    return Experiment, Activity

In [2]:
def show_results(key, username, test_horse, day, month, activity):
    database = connect(DB_NAME, USER, PASSWORD, HOST, PORT)
    Experiment, Activity = get_classes(database)

    exp_query = Experiment.select().dicts()
    act_query = Activity.select().dicts()
    if(key != None):
        exp_query = exp_query.select().where(Experiment.key == key)
        act_query = act_query.select().where(Activity.key == key)
    if(username != None):
        exp_query = exp_query.select().where(Experiment.username == username)
    if(test_horse != None):
        exp_query = exp_query.select().where(Experiment.test_horse == test_horse)
        act_query = act_query.select().where(Activity.test_horse == test_horse)
    if(day != None):
        exp_query = exp_query.select().where(Experiment.date.day == day)
    if(month != None):
        exp_query = exp_query.select().where(Experiment.date.month == month)
    if(activity != None):
        act_query = act_query.select().where(Activity.activity == activity)
    pd.options.display.float_format = '{:,.5f}'.format
    exp_querydf = pd.DataFrame.from_dict(exp_query)
    display(exp_querydf)
    act_querydf = pd.DataFrame.from_dict(act_query)
    display(act_querydf)
    

In [3]:
import ipynb.fs.full.preprocessing as preprocessing

def show_confusion_matrix_from_db(experiment_key):
    database = connect(DB_NAME, USER, PASSWORD, HOST, PORT)
    Experiment, Activity = get_classes(database)
    
    query = Experiment.select().where(Experiment.key == experiment_key).dicts()
    conf_matrix = []
    for q in query:
        conf_matrix = q
    
    if(len(conf_matrix)!=0):
        m = conf_matrix.get('confusion_matrix')
        m = m.decode()
        m = m.replace("{'confusion_matrix': ", "")
        m = m.replace("}", "")
        m = re.sub("\s+", ",", m.strip())
        m = m.replace("[,", "[")
        m = literal_eval(m)
        plt.figure(figsize=(6, 4))
        sns.heatmap(m,
                    cmap='coolwarm',
                    linecolor='white',
                    linewidths=1,
                    xticklabels=preprocessing.LABELS,
                    yticklabels=preprocessing.LABELS,
                    annot=True,
                    fmt='d')
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()